In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
import csv

In [7]:
path=r'C:\Users\shive\OneDrive\Desktop\Fake and true news data set\fake.csv'
fake=pd.read_csv(path)
fake['Check']=0
fake.head()

,title,text,subject,date,Check
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [8]:
path=r'C:\Users\shive\OneDrive\Desktop\Fake and true news data set\true.csv'
true=pd.read_csv(path)
true['Check']=1
true.head()

,title,text,subject,date,Check
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [86]:
news=pd.merge(true,fake,how='outer')
news=news.sample(frac=1).reset_index(drop=True)
news.head()

,title,text,subject,date,Check
0,Nine Republican senators urge Trump to pursue ...,WASHINGTON (Reuters) - Nine senators from his ...,politicsNews,"February 9, 2017",1
1,Hurricane Irma wreaks 'total carnage' on Barbu...,LONDON (Reuters) - The Caribbean island of Bar...,worldnews,"September 7, 2017",1
2,Trump Makes An Even Scarier Threat After Prom...,"Republican front runner Donald Trump, who rece...",News,"March 19, 2016",0
3,SARA CARTER Drops Shocker On Why A Shake-Up Is...,Sara Carter has been right on everything she s...,politics,"Dec 5, 2017",0
4,"Trump attacks Clinton on trade, says he should...","TOLEDO, Ohio; WINSTON SALEM, N.C. (Reuters) - ...",politicsNews,"October 27, 2016",1


In [88]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(news)
train, validation = train_test_split(train, test_size = 0.2)
print(len(train), len(validation), len(test) )


26938 6735 11225


In [89]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 10000
trunc_type = "post"
pad_type = "post"
oov_tok = "<OOV>"
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train.text)
word_index = tokenizer.word_index


In [90]:
training_sequences = tokenizer.texts_to_sequences(np.array(train.text))
training_padded = pad_sequences(training_sequences,truncating=trunc_type, padding=pad_type)

max_length = len(training_padded[0])

validation_sequences = tokenizer.texts_to_sequences(np.array(validation.text))
validation_padded = pad_sequences(validation_sequences, padding=pad_type, truncating=trunc_type, maxlen = max_length)

In [94]:
model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size,120, input_length=max_length),
        tf.keras.layers.Conv1D(16, 5, activation='relu'),
        tf.keras.layers.GlobalMaxPooling1D(),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

model.compile(loss='binary_crossentropy',
                      optimizer='adam', 
                      metrics=['accuracy'])

In [97]:
train_x = np.copy(training_padded)
validate_x = np.copy(validation_padded)
train_y = train['Check'].values
validate_y = validation['Check'].values

In [99]:
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self,epoch,logs={}):
            if logs.get('accuracy')>0.99:
                print('Reached 99% accuracy so cancelling training!')
                self.model.stop_training=True

In [87]:
import string
def clean_text(text):
    words = str(text).split()
    words = [i.lower() + " " for i in words]
    words = " ".join(words)
    words = words.translate(words.maketrans('', '', string.punctuation))
    return words

news['text'] = news['text'].apply(clean_text)
news.head()

,title,text,subject,date,Check
0,Nine Republican senators urge Trump to pursue ...,washington reuters nine senators from h...,politicsNews,"February 9, 2017",1
1,Hurricane Irma wreaks 'total carnage' on Barbu...,london reuters the caribbean island of ...,worldnews,"September 7, 2017",1
2,Trump Makes An Even Scarier Threat After Prom...,republican front runner donald trump who ...,News,"March 19, 2016",0
3,SARA CARTER Drops Shocker On Why A Shake-Up Is...,sara carter has been right on everything...,politics,"Dec 5, 2017",0
4,"Trump attacks Clinton on trade, says he should...",toledo ohio winston salem nc reuters u...,politicsNews,"October 27, 2016",1


In [100]:
callbacks=myCallback()
history = model.fit(train_x, train_y, epochs = 30, validation_data = (validate_x, validate_y),
                   callbacks=[callbacks])

Epoch 1/30
842/842 [==============================] - 492s 584ms/step - loss: 0.0025 - accuracy: 0.9993 - val_loss: 0.0057 - val_accuracy: 0.9979


In [109]:
t=len(news.text[67])

In [112]:
traian

8122

8122